# Web scraping: headers, the networks tab and parsing an API URL
## Helpful links and resources
- [urllib](https://docs.python.org/3/library/urllib.parse.html#) is a Python library that will pick apart URLs
- [Sessions object - request library](https://docs.python-requests.org/en/master/user/advanced/#session-objects)

In [3]:
#import libraries
import pandas as pd
import requests
import json
from urllib.parse import urlparse, parse_qs

## The networks tab
### Static data files
[Covid cases in the US - New York Times](https://www.nytimes.com/interactive/2021/us/covid-cases.html)

In [2]:
# get static data file
covid_cases_r = requests.get('https://static01.nyt.com/newsgraphics/2021/coronavirus-tracking/data/pages/usa/data.json')

In [3]:
covid_cases = covid_cases_r.json()

In [5]:
covid_cases.keys()

dict_keys(['updated', 'updated_datetime', 'location', 'counties', 'states', 'clusters', 'page_notes', 'headline_override'])

In [9]:
covid_cases_df = pd.DataFrame(covid_cases['states'])

In [11]:
covid_cases_df.head()

,country,display_name,nyt_abbr,geoid,href,latest,long_name,percent_change_14day,population,hospital_area_population,region,region_type,slug,state,subregion,date,population_adjustment
0,United States,Alabama,Ala.,USA-01,https://www.nytimes.com/interactive/2021/us/al...,"{'total': {'cases': 551298, 'deaths': 11358}, ...",Alabama,"{'raw': {'cases': 2.2488755622188883, 'deaths'...",4903185,4903185,Alabama,state,us/alabama-covid-cases,"{'us_state_fips': '01', 'name': 'Alabama', 'sh...",,2021-07-04,NaN
1,United States,Alaska,Alaska,USA-02,https://www.nytimes.com/interactive/2021/us/al...,"{'total': {'cases': 70774, 'deaths': 360}, 'av...",Alaska,"{'raw': {'cases': 31.36363636363637, 'deaths':...",731545,731545,Alaska,state,us/alaska-covid-cases,"{'us_state_fips': '02', 'name': 'Alaska', 'sho...",,2021-07-04,NaN
2,United States,Arizona,Ariz.,USA-04,https://www.nytimes.com/interactive/2021/us/ar...,"{'total': {'cases': 896960, 'deaths': 17979}, ...",Arizona,"{'raw': {'cases': 28.168067226890734, 'deaths'...",7278717,7278717,Arizona,state,us/arizona-covid-cases,"{'us_state_fips': '04', 'name': 'Arizona', 'sh...",,2021-07-04,NaN
3,United States,Arkansas,Ark.,USA-05,https://www.nytimes.com/interactive/2021/us/ar...,"{'total': {'cases': 350579, 'deaths': 5913}, '...",Arkansas,"{'raw': {'cases': 102.62035344302257, 'deaths'...",3017804,3017804,Arkansas,state,us/arkansas-covid-cases,"{'us_state_fips': '05', 'name': 'Arkansas', 's...",,2021-07-04,NaN
4,United States,California,Calif.,USA-06,https://www.nytimes.com/interactive/2021/us/ca...,"{'total': {'cases': 3821453, 'deaths': 63256},...",California,"{'raw': {'cases': 29.026748646802258, 'deaths'...",39512223,39512223,California,state,us/california-covid-cases,"{'us_state_fips': '06', 'name': 'California', ...",,2021-07-04,NaN


### Washington Post USPS data
[Story link](https://www.washingtonpost.com/business/interactive/2021/dejoy-usps-delays-by-zip-code-map/?utm_source=twitter&utm_campaign=wp_graphics&utm_medium=social)

In [12]:
df = pd.read_csv('https://www.washingtonpost.com/business/interactive/2021/dejoy-usps-delays-by-zip-code-map/usps.csv')

In [13]:
df.head(10)

,zip3,avg_days_cur,avg_days_new,avg_days_diff
0,5,2.665517,3.103653,-0.438135
1,10,2.420418,2.778212,-0.357794
2,11,2.411635,2.745268,-0.333633
3,12,2.439943,2.809153,-0.369210
4,13,2.422443,2.835077,-0.412634
5,14,2.437847,2.854191,-0.416344
6,15,2.421166,2.815743,-0.394577
7,16,2.446946,2.885994,-0.439048
8,17,2.426084,2.863679,-0.437595
9,18,2.450887,2.838465,-0.387578


### "Secret" APIs
Shopping websites are good candidates for secret APIs, such as [Target](www.target.com)

Goal: Identify prices and ratings of the first 24 results that appear when searching for paper cups on today's date, July 7.

1. Get these results with the first endpoint (search: plp_search_v1)
1. Take product IDs from plp_search_v1 results and plug them into a new endpoint

#### Target's Search API

In [1]:
# search for an item with the networks tab open to ID which APIs you can use
# this URL came from Networks -> XHR -> Headers -> Request URL
target_search_url = 'https://redsky.target.com/redsky_aggregations/v1/web/plp_search_v1?key=ff457966e64d5e877fdbad070f276d18ecec4a01&channel=WEB&count=24&default_purchasability_filter=true&include_sponsored=true&keyword=paper+plates&offset=0&page=%2Fs%2Fpaper+plates&platform=desktop&pricing_store_id=2850&scheduled_delivery_store_id=2850&store_ids=2850%2C1849%2C3284%2C3229%2C3249&useragent=Mozilla%2F5.0+%28Macintosh%3B+Intel+Mac+OS+X+10_15_7%29+AppleWebKit%2F537.36+%28KHTML%2C+like+Gecko%29+Chrome%2F91.0.4472.114+Safari%2F537.36&visitor_id=017A71BED83F0201BCBD154FC5FC4C74'

In [4]:
# parse the URL so it's easier to read
parsed_url = urlparse(target_search_url)

In [5]:
# check the parsed URL
parsed_url

ParseResult(scheme='https', netloc='redsky.target.com', path='/redsky_aggregations/v1/web/plp_search_v1', params='', query='key=ff457966e64d5e877fdbad070f276d18ecec4a01&channel=WEB&count=24&default_purchasability_filter=true&include_sponsored=true&keyword=paper+plates&offset=0&page=%2Fs%2Fpaper+plates&platform=desktop&pricing_store_id=2850&scheduled_delivery_store_id=2850&store_ids=2850%2C1849%2C3284%2C3229%2C3249&useragent=Mozilla%2F5.0+%28Macintosh%3B+Intel+Mac+OS+X+10_15_7%29+AppleWebKit%2F537.36+%28KHTML%2C+like+Gecko%29+Chrome%2F91.0.4472.114+Safari%2F537.36&visitor_id=017A71BED83F0201BCBD154FC5FC4C74', fragment='')

In [6]:
# format the endpoint and parameters
target_search_endpoint = parsed_url[0] + '://' + parsed_url[1] + parsed_url[2]

In [7]:
# https://redsky.target.com/redsky_aggregations/v1/web/plp_search_v1
target_search_endpoint

'https://redsky.target.com/redsky_aggregations/v1/web/plp_search_v1'

In [8]:
# key value pairs are separated by "&"
target_search_parameters = {}
for key_value in parsed_url[4].split('&'):
    key_value_list = key_value.split('=')
    target_search_parameters[key_value_list[0]] = key_value_list[1]

In [9]:
target_search_parameters

{'key': 'ff457966e64d5e877fdbad070f276d18ecec4a01',
 'channel': 'WEB',
 'count': '24',
 'default_purchasability_filter': 'true',
 'include_sponsored': 'true',
 'keyword': 'paper+plates',
 'offset': '0',
 'page': '%2Fs%2Fpaper+plates',
 'platform': 'desktop',
 'pricing_store_id': '2850',
 'scheduled_delivery_store_id': '2850',
 'store_ids': '2850%2C1849%2C3284%2C3229%2C3249',
 'useragent': 'Mozilla%2F5.0+%28Macintosh%3B+Intel+Mac+OS+X+10_15_7%29+AppleWebKit%2F537.36+%28KHTML%2C+like+Gecko%29+Chrome%2F91.0.4472.114+Safari%2F537.36',
 'visitor_id': '017A71BED83F0201BCBD154FC5FC4C74'}

In [10]:
# alternative way to break down a query
target_search_parameters = parse_qs(parsed_url[4])

In [11]:
target_search_parameters

{'key': ['ff457966e64d5e877fdbad070f276d18ecec4a01'],
 'channel': ['WEB'],
 'count': ['24'],
 'default_purchasability_filter': ['true'],
 'include_sponsored': ['true'],
 'keyword': ['paper plates'],
 'offset': ['0'],
 'page': ['/s/paper plates'],
 'platform': ['desktop'],
 'pricing_store_id': ['2850'],
 'scheduled_delivery_store_id': ['2850'],
 'store_ids': ['2850,1849,3284,3229,3249'],
 'useragent': ['Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'],
 'visitor_id': ['017A71BED83F0201BCBD154FC5FC4C74']}

In [12]:
target_search_r = requests.get(target_search_endpoint, params=target_search_parameters)

In [14]:
# target_search_r.json()

In [17]:
target_search_r.json()['data']['search']['products'][0]

{'__typename': 'ProductSummary',
 'tcin': '75666853',
 'original_tcin': '75666853',
 'item': {'relationship_type': 'Stand Alone',
  'relationship_type_code': 'SA',
  'merchandise_classification': {'class_id': 5, 'department_id': 253},
  'eligibility_rules': {'add_on': {'is_active': True},
   'scheduled_delivery': {'is_active': True}},
  'enrichment': {'buy_url': 'https://www.target.com/p/line-plaid-paper-plate-8-5-34-90ct-up-38-up-8482/-/A-75666853',
   'images': {'primary_image_url': 'https://target.scene7.com/is/image/Target/GUEST_39b91919-bb96-44a4-a419-2257cfd40fc5',
    'alternate_image_urls': ['https://target.scene7.com/is/image/Target/GUEST_8ffaba71-1687-4107-9e3d-c49036c358ed']}},
  'dpci': '253-05-0356',
  'cart_add_on_threshold': 35.0,
  'product_description': {'title': 'Line Plaid Paper Plate 8.5&#34; - 90ct - up &#38; up&#8482;',
   'bullet_descriptions': ['<B>Features:</B> Round (shape)',
    '<B>Dimensions (Overall):</B> 8.55 Inches (L), 8.55 Inches (W)',
    '<B>Package 

In [18]:
# change something in the parameters (like keyword)
target_search_parameters['keyword'] = 'paper+cups'
target_search_parameters

{'key': ['ff457966e64d5e877fdbad070f276d18ecec4a01'],
 'channel': ['WEB'],
 'count': ['24'],
 'default_purchasability_filter': ['true'],
 'include_sponsored': ['true'],
 'keyword': 'paper+cups',
 'offset': ['0'],
 'page': ['/s/paper plates'],
 'platform': ['desktop'],
 'pricing_store_id': ['2850'],
 'scheduled_delivery_store_id': ['2850'],
 'store_ids': ['2850,1849,3284,3229,3249'],
 'useragent': ['Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'],
 'visitor_id': ['017A71BED83F0201BCBD154FC5FC4C74']}

In [19]:
target_search_r = requests.get(target_search_endpoint, params=target_search_parameters)

In [20]:
target_search_r.json()['data']['search']['products'][0]

{'__typename': 'ProductSummary',
 'tcin': '12970172',
 'original_tcin': '12970172',
 'item': {'relationship_type': 'Stand Alone',
  'relationship_type_code': 'SA',
  'merchandise_classification': {'class_id': 5, 'department_id': 253},
  'eligibility_rules': {'add_on': {'is_active': True},
   'scheduled_delivery': {'is_active': True}},
  'enrichment': {'buy_url': 'https://www.target.com/p/dixie-everyday-assorted-designs-cold-cups-54ct-9oz/-/A-12970172',
   'images': {'primary_image_url': 'https://target.scene7.com/is/image/Target/GUEST_9b4d6235-6e2e-4c7c-a541-6babba8eec63',
    'alternate_image_urls': ['https://target.scene7.com/is/image/Target/GUEST_d2d66bf2-7f60-444c-a286-16f29b008b5f',
     'https://target.scene7.com/is/image/Target/GUEST_4f0f4c6a-4310-4db5-b9ff-f5ef64671679',
     'https://target.scene7.com/is/image/Target/GUEST_5e6b372c-c099-4e8b-93b0-8611f6580497']},
   'videos': [{'is_list_page_eligible': False,
     'video_files': [{'mime_type': 'video/mp4',
       'video_url': 

In [33]:
target_search_products = target_search_r.json()['data']['search']['products']

In [22]:
# get request with endpoint and params
target_fulfillment_url = 'https://redsky.target.com/redsky_aggregations/v1/web/plp_fulfillment_v1?key=ff457966e64d5e877fdbad070f276d18ecec4a01&tcins=13025050%2C17316119%2C78327970%2C13025048%2C53413908%2C78327963%2C75564425%2C14477565%2C78327952%2C13356889%2C16889463%2C78327958%2C75661937%2C47021025%2C18813263%2C75661939%2C14762347%2C14477283%2C76078874%2C50335649%2C13937463%2C76150031%2C47981892%2C13636299%2C13162871%2C75564405%2C52297080%2C50335768&store_id=2850&zip=11201&state=NY&latitude=40.690&longitude=-74.000&scheduled_delivery_store_id=2850'
target_client_url = 'https://redsky.target.com/redsky_aggregations/v1/web/pdp_client_v1?key=ff457966e64d5e877fdbad070f276d18ecec4a01&tcin=12970172&store_id=2850&has_store_id=true&pricing_store_id=2850&has_pricing_store_id=true&scheduled_delivery_store_id=2850&has_scheduled_delivery_store_id=true&has_financing_options=true&visitor_id=017A71BED83F0201BCBD154FC5FC4C74&has_size_context=true'

In [23]:
# parse this url
parsed_url = urlparse(target_client_url)
parsed_url

ParseResult(scheme='https', netloc='redsky.target.com', path='/redsky_aggregations/v1/web/pdp_client_v1', params='', query='key=ff457966e64d5e877fdbad070f276d18ecec4a01&tcin=12970172&store_id=2850&has_store_id=true&pricing_store_id=2850&has_pricing_store_id=true&scheduled_delivery_store_id=2850&has_scheduled_delivery_store_id=true&has_financing_options=true&visitor_id=017A71BED83F0201BCBD154FC5FC4C74&has_size_context=true', fragment='')

In [24]:
target_client_endpoint = parsed_url[0] + '://' + parsed_url[1] + parsed_url[2]

In [25]:
target_client_endpoint

'https://redsky.target.com/redsky_aggregations/v1/web/pdp_client_v1'

In [26]:
target_client_params = parse_qs(parsed_url[4])

In [28]:
target_client_params

{'key': ['ff457966e64d5e877fdbad070f276d18ecec4a01'],
 'tcin': ['12970172'],
 'store_id': ['2850'],
 'has_store_id': ['true'],
 'pricing_store_id': ['2850'],
 'has_pricing_store_id': ['true'],
 'scheduled_delivery_store_id': ['2850'],
 'has_scheduled_delivery_store_id': ['true'],
 'has_financing_options': ['true'],
 'visitor_id': ['017A71BED83F0201BCBD154FC5FC4C74'],
 'has_size_context': ['true']}

In [29]:
# drill down the json file
# key value pairs are separated by "&"
# def parse_target_url(url):
#     params = {}
#     for key_value in url[4].split('&'):
#         key_value_list = key_value.split('=')
#         params[key_value_list[0]] = key_value_list[1]
#     return params

In [30]:
# target_aggregate_params = parse_target_url(parsed_url)

In [31]:
# target_aggregate_params

In [32]:
# target_aggregate_endpoint = parsed_url[0] + '://' + parsed_url[1] + parsed_url[2]

In [38]:
target_search_products[1]['tcin']

'75557224'

In [39]:
target_client_params['tcin'] = ['75557224']

In [40]:
target_client_params

{'key': ['ff457966e64d5e877fdbad070f276d18ecec4a01'],
 'tcin': ['75557224'],
 'store_id': ['2850'],
 'has_store_id': ['true'],
 'pricing_store_id': ['2850'],
 'has_pricing_store_id': ['true'],
 'scheduled_delivery_store_id': ['2850'],
 'has_scheduled_delivery_store_id': ['true'],
 'has_financing_options': ['true'],
 'visitor_id': ['017A71BED83F0201BCBD154FC5FC4C74'],
 'has_size_context': ['true']}

In [41]:
target_client_r = requests.get(target_client_endpoint, params=target_client_params)

In [45]:
# target_client_r.json()
target_paper_cups = []
for product in target_search_products:
    target_client_params['tcin'] = product['tcin']
    target_client_r = requests.get(target_client_endpoint, params=target_client_params)
    target_paper_cups.append(target_client_r.json())

In [55]:
target_paper_cups_2 = []
for item in target_paper_cups:
    target_paper_cups_2.append(item['data']['product'])

In [58]:
target_df = pd.DataFrame(target_paper_cups_2)

In [59]:
target_df.head()

,__typename,tcin,ratings_and_reviews,financing_options,item,price,promotions,variation_hierarchy,children
0,Product,12970172,"{'has_verified': True, 'statistics': {'not_rec...","{'channel': 'DIGITAL', 'providers': [{'finance...","{'dpci': '253-05-0076', 'relationship_type_cod...","{'current_retail': 3.89, 'formatted_current_pr...",[],NaN,NaN
1,Product,75557224,"{'has_verified': True, 'statistics': {'not_rec...","{'channel': 'DIGITAL', 'providers': [{'finance...","{'dpci': '253-05-0032', 'relationship_type_cod...","{'current_retail': 2.99, 'formatted_current_pr...",[],NaN,NaN
2,Product,82242697,"{'has_verified': True, 'statistics': {'not_rec...","{'channel': 'DIGITAL', 'providers': [{'finance...","{'dpci': '253-05-0166', 'relationship_type_cod...","{'current_retail': 6.79, 'formatted_current_pr...",[],NaN,NaN
3,Product,79620967,"{'has_verified': True, 'statistics': {'not_rec...",NaN,"{'relationship_type_code': 'VAP', 'cart_add_on...","{'current_retail_min': 2.49, 'formatted_curren...",[],"[{'name': 'Size', 'value': '72ct', 'tcin': '14...","[{'__typename': 'Product', 'tcin': '14736272',..."
4,Product,75557230,"{'has_verified': True, 'statistics': {'not_rec...","{'channel': 'DIGITAL', 'providers': [{'finance...","{'dpci': '253-05-0040', 'relationship_type_cod...","{'current_retail': 2.99, 'formatted_current_pr...",[],NaN,NaN


In [192]:
# drill down some more

In [ ]:
selected_item_urls = ['/p/casabella-infuse-bathroom-microfiber-cloths-2pk/-/A-78327970']

#### Target's aggregation API

In [195]:
# parse the URL so it's easier to read
target_list = urlparse('https://redsky.target.com/redsky_aggregations/v1/web/plp_fulfillment_v1?key=ff457966e64d5e877fdbad070f276d18ecec4a01&tcins=81107269%2C81068829%2C14135567%2C81068792%2C82079503%2C81829962%2C81068790%2C81506339%2C80935950%2C81107259%2C81068797%2C11069188%2C81506334%2C81107271%2C81068773%2C81180792%2C81107267%2C81068789%2C81068796%2C81506336%2C81107268%2C81068821%2C81564691%2C81953908%2C81068815%2C81068825%2C81068787%2C81564688&store_id=2850&zip=11201&state=NY&latitude=40.690&longitude=-74.000&scheduled_delivery_store_id=2850')

In [196]:
# check the parsed URL
target_list

ParseResult(scheme='https', netloc='redsky.target.com', path='/redsky_aggregations/v1/web/plp_fulfillment_v1', params='', query='key=ff457966e64d5e877fdbad070f276d18ecec4a01&tcins=81107269%2C81068829%2C14135567%2C81068792%2C82079503%2C81829962%2C81068790%2C81506339%2C80935950%2C81107259%2C81068797%2C11069188%2C81506334%2C81107271%2C81068773%2C81180792%2C81107267%2C81068789%2C81068796%2C81506336%2C81107268%2C81068821%2C81564691%2C81953908%2C81068815%2C81068825%2C81068787%2C81564688&store_id=2850&zip=11201&state=NY&latitude=40.690&longitude=-74.000&scheduled_delivery_store_id=2850', fragment='')

In [197]:
# format the endpoint and parameters
target_list_endpoint = target_list[0] + '://' + target_list[1] + target_list[2]
target_list_params = {}
for parameter in target_list[4].split('&'):
    key_value = parameter.split('=')
    target_list_params[key_value[0]] = key_value[1]

In [198]:
# change something in the parameters (like tcins)
target_list_params['tcins'] = '81107269'

In [199]:
# get request with endpoint and params
target_list_r = requests.get(target_list_endpoint, params=target_list_params)

In [201]:
# drill down the json file
target_list_r.json()['data']['product_summaries']

[{'__typename': 'ProductSummary',
  'tcin': '81107269',
  'fulfillment': {'product_id': '81107269',
   'is_out_of_stock_in_all_store_locations': False,
   'shipping_options': {'availability_status': 'IN_STOCK',
    'loyalty_availability_status': 'IN_STOCK',
    'available_to_promise_quantity': 399.0,
    'minimum_order_quantity': 1.0,
    'services': [{'shipping_method_id': 'STANDARD',
      'min_delivery_date': '2021-07-08',
      'max_delivery_date': '2021-07-08',
      'is_two_day_shipping': True,
      'is_base_shipping_method': True,
      'service_level_description': '2-day shipping',
      'shipping_method_short_description': 'Standard',
      'cutoff': '2021-07-05T16:00:00Z'}]},
   'store_options': [{'location_name': 'Brooklyn Fulton St',
     'location_address': '445 Albee Square West,BROOKLYN,NY,11201-3016',
     'location_id': '2850',
     'search_response_store_type': 'PRIMARY',
     'order_pickup': {'availability_status': 'UNAVAILABLE',
      'reason_code': 'IN_ELIGIBLE'},

In [203]:
# drill down some more
target_list_r.json()['data']['product_summaries'][0]

{'__typename': 'ProductSummary',
 'tcin': '81107269',
 'fulfillment': {'product_id': '81107269',
  'is_out_of_stock_in_all_store_locations': False,
  'shipping_options': {'availability_status': 'IN_STOCK',
   'loyalty_availability_status': 'IN_STOCK',
   'available_to_promise_quantity': 399.0,
   'minimum_order_quantity': 1.0,
   'services': [{'shipping_method_id': 'STANDARD',
     'min_delivery_date': '2021-07-08',
     'max_delivery_date': '2021-07-08',
     'is_two_day_shipping': True,
     'is_base_shipping_method': True,
     'service_level_description': '2-day shipping',
     'shipping_method_short_description': 'Standard',
     'cutoff': '2021-07-05T16:00:00Z'}]},
  'store_options': [{'location_name': 'Brooklyn Fulton St',
    'location_address': '445 Albee Square West,BROOKLYN,NY,11201-3016',
    'location_id': '2850',
    'search_response_store_type': 'PRIMARY',
    'order_pickup': {'availability_status': 'UNAVAILABLE',
     'reason_code': 'IN_ELIGIBLE'},
    'in_store_only': 

## Using sessions to login
### Accessing password-protected pages
[Sessions object - request library](https://docs.python-requests.org/en/master/user/advanced/#session-objects)

In [60]:
# open up a session so that your login credentials are saved
session = requests.Session()

In [61]:
# load in config file with passwords
with open('../config/config.json') as json_file:
    config = json.load(json_file)

In [62]:
# check the website for the login parameters
atom_signin_endpoint = 'https://atom.finance/session/signin'

In [63]:
atom_signin_payload = {
    'username': 'katiemarriner',
    'password': config['atom_password']
}

In [64]:
# post the payload to the site to login with the correct log in endpoint
s = session.post(atom_signin_endpoint, data=atom_signin_payload)

In [65]:
s.text

'{"success":true,"userId":"70920d34-5ae8-4bd3-81d0-f0ce721a1095","email":"kemarriner@gmail.com","username":"katiemarriner","firstName":"Katie","lastName":"Marriner","name":"Katie Marriner","inviteCode":null}'

In [66]:
# check credentials to see if successful
interday_endpoint = 'https://atom.finance/graphql'

In [71]:
interday_payload = {
    'operationName': "getChartInterday",
    'query': "query getChartInterday($symbol: String!, $chartType: String, $rangeUnit: String, $rangeCount: Int) {\n  interday(symbol: $symbol, chartType: $chartType, rangeUnit: $rangeUnit, rangeCount: $rangeCount) {\n    date\n    price\n    volume\n    changePercent\n    open\n    close\n    high\n    low\n    __typename\n  }\n}\n",
    'variables': {'symbol': "SPY"}
}

In [175]:
# look at an example page to get you started with a query

In [74]:
# create a new post object from the example
s = session.post(interday_endpoint, json=interday_payload)

In [76]:
s.json()

{'data': {'interday': [{'date': 1049227200000,
    'price': 86.04,
    'volume': 53821200,
    'changePercent': None,
    'open': None,
    'close': None,
    'high': 86.39,
    'low': 84.91,
    '__typename': 'ChartData'},
   {'date': 1049313600000,
    'price': 88.12,
    'volume': 50500600,
    'changePercent': None,
    'open': None,
    'close': None,
    'high': 88.77,
    'low': 87.5,
    '__typename': 'ChartData'},
   {'date': 1049400000000,
    'price': 87.7,
    'volume': 49012500,
    'changePercent': None,
    'open': None,
    'close': None,
    'high': 88.99,
    'low': 87.65,
    '__typename': 'ChartData'},
   {'date': 1049486400000,
    'price': 88.22,
    'volume': 36261800,
    'changePercent': None,
    'open': None,
    'close': None,
    'high': 88.59,
    'low': 87.62,
    '__typename': 'ChartData'},
   {'date': 1049745600000,
    'price': 88.05,
    'volume': 69824800,
    'changePercent': None,
    'open': None,
    'close': None,
    'high': 90.85,
    'low': 8

In [ ]:
# post request for the data

In [179]:
# check to see what is returned

'/bundles/web/images/user-image.007dad08.svg'